**Imports**

In [1]:
import tarfile
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from skimage.util import montage 
from skimage.transform import rotate
from datetime import datetime
import seaborn as sns
import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
#from tensorflow.keras.layers.experimental import preprocessing
import cv2
import os
from dotenv import load_dotenv
import wandb
from wandb.keras import WandbCallback
load_dotenv()

2022-11-30 19:11:22.040276: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 19:11:22.337154: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-30 19:11:23.058849: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/viktorsalling/miniconda3/lib/
2022-11-30 19:11:23.059596: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin

True

**Unpack data** 

*NB: Size is 13.5 GB, code is commented out to prevent accidental unpacking*

**TEST THIS BEFORE HAND IN**

In [ ]:
#file = tarfile.open('../input/brats-2021-task1/BraTS2021_Training_Data.tar')

#file.extractall(os.getenv('TRAIN-PATH'))
#file.close()

**Metrics and loss**

In [2]:
def dice_coef(y_true, y_pred, epsilon=0.00001):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
        =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    
    """
    axis = (0,1,2,3)
    dice_numerator = 2. * K.sum(y_true * y_pred, axis=axis) + epsilon
    dice_denominator = K.sum(y_true*y_true, axis=axis) + K.sum(y_pred*y_pred, axis=axis) + epsilon
    return K.mean((dice_numerator)/(dice_denominator))

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

# Loss functions here: https://www.kaggle.com/code/bigironsphere/loss-function-library-keras-pytorch/notebook

ALPHA = 0.8
GAMMA = 2

def focal_loss(targets, inputs, alpha=ALPHA, gamma=GAMMA):    
    
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    
    BCE = K.binary_crossentropy(targets, inputs)
    BCE_EXP = K.exp(-BCE)
    focal_loss = K.mean(alpha * K.pow((1-BCE_EXP), gamma) * BCE)
    
    return focal_loss



# define per class evaluation of dice coef
# inspired by https://github.com/keras-team/keras/issues/9395
def dice_coef_healthy(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true[0,:,:,:,0] * y_pred[0,:,:,:,0]))
    return (2. * intersection) / (K.sum(K.square(y_true[0,:,:,:,0])) + K.sum(K.square(y_pred[0,:,:,:,0])) + epsilon)

def dice_coef_necrotic(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true[0,:,:,:,1] * y_pred[0,:,:,:,1]))
    return (2. * intersection) / (K.sum(K.square(y_true[0,:,:,:,1])) + K.sum(K.square(y_pred[0,:,:,:,1])) + epsilon)

def dice_coef_edema(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true[0,:,:,:,2] * y_pred[0,:,:,:,2]))
    return (2. * intersection) / (K.sum(K.square(y_true[0,:,:,:,2])) + K.sum(K.square(y_pred[0,:,:,:,2])) + epsilon)

def dice_coef_enhancing(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true[0,:,:,:,3] * y_pred[0,:,:,:,3]))
    return (2. * intersection) / (K.sum(K.square(y_true[0,:,:,:,3])) + K.sum(K.square(y_pred[0,:,:,:,3])) + epsilon)



# Computing Precision 
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    
# Computing Sensitivity      
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())


# Computing Specificity
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

**Data constants**

In [8]:
IMG_SIZE=128 # Originally 240x240, we reduce this to reduce memory usage
SLICES=128 # Originally 155, reduced for same reason. Must be a power of 2 (or at least divisible by 2, 3 times)
SLICES_START=13 # We skip the first (and the last) few slices as they should contain less information
BATCH_SIZE=1
CHANNELS=4

TRAIN_DATASET_PATH = os.getenv('TRAIN-PATH')

**Data generation**

In [9]:
data_directories = [f.path for f in os.scandir(TRAIN_DATASET_PATH) if f.is_dir()]

def pathListIntoIds(dirList):
    x = []
    for i in range(0,len(dirList)):
        x.append(dirList[i][dirList[i].rfind('/')+1:])
    return x

data_ids = pathListIntoIds(data_directories); 

    
train_and_test_ids, val_ids = train_test_split(data_ids,test_size=0.2) 
train_ids, test_ids = train_test_split(train_and_test_ids,test_size=0.2)


keras = tf.compat.v1.keras
Sequence = keras.utils.Sequence

class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, dim=(IMG_SIZE,IMG_SIZE), batch_size = BATCH_SIZE, n_channels = CHANNELS, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        Batch_ids = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(Batch_ids)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, Batch_ids):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size*SLICES, *self.dim, self.n_channels))
        y = np.zeros((self.batch_size*SLICES, IMG_SIZE, IMG_SIZE))

        
        # Generate data
        for c, i in enumerate(Batch_ids):
            case_path = os.path.join(TRAIN_DATASET_PATH, i)

            data_path = os.path.join(case_path, f'{i}_flair.nii.gz');
            flair = nib.load(data_path).get_fdata()
            
            data_path = os.path.join(case_path, f'{i}_t1ce.nii.gz');
            ce = nib.load(data_path).get_fdata()   
            
            data_path = os.path.join(case_path, f'{i}_t1.nii.gz');
            t1 = nib.load(data_path).get_fdata()       

            data_path = os.path.join(case_path, f'{i}_t2.nii.gz');
            t2 = nib.load(data_path).get_fdata()
            
            data_path = os.path.join(case_path, f'{i}_seg.nii.gz');
            seg = nib.load(data_path).get_fdata()
        
            for j in range(SLICES):
                X[j+(SLICES*c),:,:,0] = cv2.resize(flair[:,:,j+SLICES_START], (IMG_SIZE, IMG_SIZE))
                X[j+(SLICES*c),:,:,1] = cv2.resize(ce[:,:,j+SLICES_START], (IMG_SIZE, IMG_SIZE))
                X[j+(SLICES*c),:,:,2] = cv2.resize(t1[:,:,j+SLICES_START], (IMG_SIZE, IMG_SIZE))
                X[j+(SLICES*c),:,:,3] = cv2.resize(t2[:,:,j+SLICES_START], (IMG_SIZE, IMG_SIZE))                
                
                y[j +SLICES*c,:,:] = cv2.resize(seg[:,:,j+SLICES_START], (IMG_SIZE, IMG_SIZE))

        X = X.reshape(1,SLICES,IMG_SIZE,IMG_SIZE, CHANNELS)
        y = y.reshape(1,SLICES,IMG_SIZE,IMG_SIZE)
        # Generate masks
        #y[y==4] = 3;
        y = tf.one_hot(y, 4);

        # Avoid dividing by zero - return early
        if np.max(X) == 0.0:
          return np.zeros(X.shape), y

        return X/np.max(X), y
        
training_generator = DataGenerator(train_ids)
valid_generator = DataGenerator(val_ids)
test_generator = DataGenerator(test_ids)

**Baseline model**

In [10]:
def unet_3d_conv(layer, filters):
    layer = Conv3D(filters, kernel_size=(3,3,3), strides=(1,1,1), padding='same')(layer)
    layer = BatchNormalization()(layer)
    layer = Activation('relu')(layer)
    return layer


def unet_3d(input_img):
    c1 = unet_3d_conv(input_img, 8)
    c2 = unet_3d_conv(c1, 16)
    
    c3 = MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2))(c2)
    

    c4 = unet_3d_conv(c3, 16)
    c5 = unet_3d_conv(c4, 32)
    c6 = MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2))(c5)

    c7 = unet_3d_conv(c6, 32)
    c8 = unet_3d_conv(c7, 64)
    c9 = MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2))(c8)

    c10 = unet_3d_conv(c9, 64)
    c11 = unet_3d_conv(c10, 128)
    c12 = UpSampling3D(2)(c11)

    c13 = concatenate([c8, c12])
    c14 = unet_3d_conv(c13, 64)
    c15 = unet_3d_conv(c14, 64)
    c16 = UpSampling3D(2)(c15)

    c17 = concatenate([c5, c16])
    c18 = unet_3d_conv(c17, 32)
    c19 = unet_3d_conv(c18, 32)
    c20 = UpSampling3D(2)(c19)

    c21 = concatenate([c2, c20])
    c22 = unet_3d_conv(c21, 16)
    c23 = unet_3d_conv(c22, 16)
    c24 = Conv3D(4, kernel_size=(1,1,1), strides=(1,1,1), padding='same')(c23)
    c25 = Activation('softmax')(c24)


    model = Model(inputs=input_img, outputs=c25)
    return model 


input_layer = Input((SLICES, IMG_SIZE, IMG_SIZE, CHANNELS))
model = unet_3d(input_layer) 

**Train and test (baseline) model**

In [11]:
LR = 0.01
EPOCHS = 20

wandb.init(project="BraTS2021", entity="vj-dl")
wandb.config = {
  "learning_rate": LR,
  "epochs": EPOCHS,
  "batch_size": BATCH_SIZE,
  "img_size": IMG_SIZE, 
  "slices": SLICES
}

model.compile(optimizer=keras.optimizers.SGD(learning_rate=LR), loss="categorical_crossentropy", metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), dice_coef, precision, sensitivity, specificity, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing, dice_coef_healthy])
model.summary()
model.fit(training_generator, epochs=EPOCHS, validation_data=valid_generator, callbacks=[WandbCallback()])
model.save("baseline_model.h5")


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                28, 4)]                                                           
                                                                                                  
 conv3d_30 (Conv3D)             (None, 128, 128, 12  872         ['input_3[0][0]']                
                                8, 8)                                                             
                                                                                                  
 batch_normalization_28 (BatchN  (None, 128, 128, 12  32         ['conv3d_30[0][0]']              
 ormalization)                  8, 8)                                                       

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/20
800/800 [==============================] - ETA: 0s - loss: 0.1014 - accuracy: 0.9869 - mean_io_u_1: 0.3753 - dice_coef: 0.4100 - precision: 0.9912 - sensitivity: 0.9742 - specificity: 0.9974 - dice_coef_necrotic: 0.1634 - dice_coef_edema: 0.4151 - dice_coef_enhancing: 0.0748 - dice_coef_healthy: 0.9866

INFO:tensorflow:Assets written to: /home/viktorsalling/Documents/Repositories/BraTS2021/wandb/run-20221130_191256-2omzn00z/files/model-best/assets


INFO:tensorflow:Assets written to: /home/viktorsalling/Documents/Repositories/BraTS2021/wandb/run-20221130_191256-2omzn00z/files/model-best/assets
wandb: Adding directory to artifact (/home/viktorsalling/Documents/Repositories/BraTS2021/wandb/run-20221130_191256-2omzn00z/files/model-best)... Done. 0.0s


...

INFO:tensorflow:Assets written to: /home/viktorsalling/Documents/Repositories/BraTS2021/wandb/run-20221130_191256-2omzn00z/files/model-best/assets
wandb: Adding directory to artifact (/home/viktorsalling/Documents/Repositories/BraTS2021/wandb/run-20221130_191256-2omzn00z/files/model-best)... Done. 0.0s


800/800 [==============================] - 538s 672ms/step - loss: 0.0213 - accuracy: 0.9922 - mean_io_u_1: 0.3753 - dice_coef: 0.4857 - precision: 0.9942 - sensitivity: 0.9921 - specificity: 0.9981 - dice_coef_necrotic: 0.2693 - dice_coef_edema: 0.5726 - dice_coef_enhancing: 0.1023 - dice_coef_healthy: 0.9984 - val_loss: 0.0663 - val_accuracy: 0.9767 - val_mean_io_u_1: 0.3753 - val_dice_coef: 0.3760 - val_precision: 0.9788 - val_sensitivity: 0.9759 - val_specificity: 0.9930 - val_dice_coef_necrotic: 0.1471 - val_dice_coef_edema: 0.3113 - val_dice_coef_enhancing: 0.0547 - val_dice_coef_healthy: 0.9910
Epoch 5/20
800/800 [==============================] - 541s 676ms/step - loss: 0.0203 - accuracy: 0.9924 - mean_io_u_1: 0.3753 - dice_coef: 0.4913 - precision: 0.9943 - sensitivity: 0.9923 - specificity: 0.9981 - dice_coef_necrotic: 0.2741 - dice_coef_edema: 0.5848 - dice_coef_enhancing: 0.1078 - dice_coef_healthy: 0.9985 - val_loss: 0.0751 - val_accuracy: 0.9720 - val_mean_io_u_1: 0.3753 

INFO:tensorflow:Assets written to: /home/viktorsalling/Documents/Repositories/BraTS2021/wandb/run-20221130_191256-2omzn00z/files/model-best/assets


INFO:tensorflow:Assets written to: /home/viktorsalling/Documents/Repositories/BraTS2021/wandb/run-20221130_191256-2omzn00z/files/model-best/assets
wandb: Adding directory to artifact (/home/viktorsalling/Documents/Repositories/BraTS2021/wandb/run-20221130_191256-2omzn00z/files/model-best)... Done. 0.0s


800/800 [==============================] - 557s 696ms/step - loss: 0.0215 - accuracy: 0.9915 - mean_io_u_1: 0.3753 - dice_coef: 0.5114 - precision: 0.9948 - sensitivity: 0.9909 - specificity: 0.9983 - dice_coef_necrotic: 0.2801 - dice_coef_edema: 0.5603 - dice_coef_enhancing: 0.2070 - dice_coef_healthy: 0.9980 - val_loss: 0.0644 - val_accuracy: 0.9832 - val_mean_io_u_1: 0.3753 - val_dice_coef: 0.3525 - val_precision: 0.9844 - val_sensitivity: 0.9829 - val_specificity: 0.9949 - val_dice_coef_necrotic: 0.0965 - val_dice_coef_edema: 0.2793 - val_dice_coef_enhancing: 0.0415 - val_dice_coef_healthy: 0.9927


**Evaluate on test data**

In [12]:
model.evaluate(test_generator)

200/200 [==============================] - 87s 434ms/step - loss: 0.0787 - accuracy: 0.9800 - mean_io_u_1: 0.3753 - dice_coef: 0.3339 - precision: 0.9809 - sensitivity: 0.9795 - specificity: 0.9937 - dice_coef_necrotic: 0.0800 - dice_coef_edema: 0.2303 - dice_coef_enhancing: 0.0344 - dice_coef_healthy: 0.9909


[0.07869383692741394,
 0.9800161123275757,
 0.37527090311050415,
 0.3339027464389801,
 0.9809294939041138,
 0.9795030951499939,
 0.9937281012535095,
 0.0800149142742157,
 0.23031704127788544,
 0.034410323947668076,
 0.9908667802810669]

**Callbacks for improved model**

In [ ]:
callbacks = [keras.callbacks.EarlyStopping(monitor='dice_coef', min_delta=0,
                               patience=2, verbose=1, mode='auto'),
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)
    ]

**Improved model**

In [ ]:
def unet_3d_conv(layer, filters):
    layer = Conv3D(filters, kernel_size=(3,3,3), strides=(1,1,1), padding='same')(layer)
    layer = BatchNormalization()(layer)
    layer = Activation('relu')(layer)
    return layer


def unet_3d(input_img):
    c1 = unet_3d_conv(input_img, 8)
    c2 = unet_3d_conv(c1, 16)
    
    # Maybe use Convolution with stride = 2 in improved model
    #c3 = MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2))(c2)
    c3 = Conv3D(16, kernel_size=(3,3,3), strides=(2,2,2), padding='same')(c2)
    

    c4 = unet_3d_conv(c3, 16)
    c5 = unet_3d_conv(c4, 32)
    #c6 = MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2))(c5)
    c6 = Conv3D(32, kernel_size=(3,3,3), strides=(2,2,2), padding='same')(c5)

    c7 = unet_3d_conv(c6, 32)
    c8 = unet_3d_conv(c7, 64)
    #c9 = MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2))(c8)
    c9 = Conv3D(64, kernel_size=(3,3,3), strides=(2,2,2), padding='same')(c8)

    c10 = unet_3d_conv(c9, 64)
    c11 = unet_3d_conv(c10, 128)
    #c12 = Conv3DTranspose(filters=128, kernel_size=(1,1,1), strides=(2,2,2))(c11)
    c12 = UpSampling3D(2)(c11)

    c13 = concatenate([c8, c12])
    c14 = unet_3d_conv(c13, 64)
    c15 = unet_3d_conv(c14, 64)
    c16 = UpSampling3D(2)(c15)
    #c16 = Conv3DTranspose(filters=64, kernel_size=(1,1,1), strides=(2,2,2))(c15)

    c17 = concatenate([c5, c16])
    c18 = unet_3d_conv(c17, 32)
    c19 = unet_3d_conv(c18, 32)
    c20 = UpSampling3D(2)(c19)
    #c20 = Conv3DTranspose(filters=32, kernel_size=(1,1,1), strides=(2,2,2))(c19)

    c21 = concatenate([c2, c20])
    c22 = unet_3d_conv(c21, 16)
    c23 = unet_3d_conv(c22, 16)
    c24 = Conv3D(4, kernel_size=(1,1,1), strides=(1,1,1), padding='same')(c23)
    c25 = Activation('softmax')(c24)


    model = Model(inputs=input_img, outputs=c25)
    return model 


input_layer = Input((SLICES, IMG_SIZE, IMG_SIZE, CHANNELS))
model = unet_3d(input_layer) 

**Train and test improved model**

In [ ]:
LR = 0.01
EPOCHS = 20

wandb.init(project="BraTS2021", entity="vj-dl")
wandb.config = {
  "learning_rate": LR,
  "epochs": EPOCHS,
  "batch_size": BATCH_SIZE,
  "img_size": IMG_SIZE, 
  "slices": SLICES
}

model.compile(optimizer=keras.optimizers.SGD(learning_rate=LR, momentum=0.95), loss=focal_loss, metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), dice_coef, precision, sensitivity, specificity, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing, dice_coef_healthy])
model.summary()
model.fit(training_generator, epochs=EPOCHS, validation_data=valid_generator, callbacks=[callbacks, WandbCallback()])


**Predictions? Maybe we should do an example (or a few) where we use the trained model to predict segmentation masks on data we have not trained on (i.e. the two brain scans that were included in the data set but not in the training-data file**

**Something like this:** https://pyimagesearch.com/2022/02/21/u-net-image-segmentation-in-keras/ **(scroll down to "predictions")**